In [2]:
# Cài đặt Unsloth và các thư viện cần thiết
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install trl datasets accelerate peft bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /private/var/folders/l9/1_7ddvhx315glb_06svh5x8h0000gn/T/pip-install-cbngmruw/unsloth_52e971520262483394c21ab12219b3ae
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /private/var/folders/l9/1_7ddvhx315glb_06svh5x8h0000gn/T/pip-install-cbngmruw/unsloth_52e971520262483394c21ab12219b3ae
  Resolved https://github.com/unslothai/unsloth.git to commit bda9e3d39b425f902d29e80c1f2870be7048d9c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
ERROR: Package 'unsloth' requires a different Python: 3.14.0 not in '<3.14,>=3.9'
  Using cached trl-0.25.1-py3-none-an

In [4]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. Chọn Mô hình Cơ sở (Gemma 2B của Google)
model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Tải Mô hình và Tokenizer (áp dụng QLoRA)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. Thiết lập LoRA (Low-Rank Adaptation)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 3. Tải Dữ liệu (Dùng hàm dataset.load_dataset như cũ)
# Mày cần đảm bảo file 'dataset.jsonl' đã được tải lên Colab
dataset = load_dataset("json", data_files="dataset.jsonl", split="train")

# 4. Áp dụng Template Hội thoại và Định dạng Dữ liệu (Sử dụng hàm chuẩn)

# Định nghĩa hàm để áp dụng template hội thoại (cần dùng hàm của tokenizer)
def formatting_prompts_func(examples):
    texts = []
    for conversation in examples["messages"]:
        # Tùy chỉnh: Mày phải đảm bảo dataset của mày có key là "messages"
        # và cấu trúc bên trong là list of dicts: {"role": "...", "content": "..."}

        # Áp dụng template chat của Mistral/Gemma
        text = tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return { "text" : texts, }

# Áp dụng hàm định dạng lên Dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)

# (Đoạn code định dạng dữ liệu đã chạy thành công ở trên)

# QUAN TRỌNG: Sửa lại cách truy cập dataset
# Thay vì dataset['train'][0]['text'], mày chỉ cần dùng dataset[0]['text']
print("--- Kiểm tra mẫu dữ liệu đầu tiên sau khi định dạng ---")
# Mày chỉ cần truy cập dataset[0]['text']
print(dataset[0]['text'])
print("---------------------------------------------------------")

ModuleNotFoundError: No module named 'unsloth'

In [4]:
# 5. Thiết lập Tham số Huấn luyện
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10, # 3-5 epoch thường là đủ
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 42,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
    ),
)

# 6. Bắt đầu Huấn luyện
trainer_output = trainer.train()

print("Hoàn thành Fine-tuning!")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/21 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21 | Num Epochs = 10 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488 of 7,255,363,584 (0.19% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: holetinnghia (holetinnghia-gitlab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.043100
2,2.449700
3,2.125900
4,1.427200
5,0.784900
6,0.467500
7,0.278500
8,0.309500
9,0.176900
10,0.197000


train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/grad_norm,█▇▆▆▄▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▂▄▅▇██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁
train/loss,▇█▇▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1842470040870912.0
train/epoch,10
train/global_step,30
train/grad_norm,0.03512
train/learning_rate,1e-05
train/loss,0.1625


Hoàn thành Fine-tuning!


In [ ]:
# Hợp nhất LoRA weights vào mô hình cơ sở
model.save_pretrained_merged("gemma_taolao_merged", tokenizer, save_method = "safetensors")

# Nén thư mục lại để tải về máy Mac
!zip -r gemma_taolao_merged.zip gemma_taolao_merged

config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00003.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/3 [00:00<?, ?it/s]

...

Unsloth: Preparing safetensor model files:  33%|███▎      | 1/3 [01:12<02:24, 72.31s/it]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files:  67%|██████▋   | 2/3 [02:51<01:28, 88.39s/it]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/3 [00:00<?, ?it/s]